<a href="https://colab.research.google.com/github/Pratik-Nikam/MachineLearning/blob/main/T5-HDFC-FAQ-Generative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [17]:
import json
with open('/content/sample_data/HDFC_Faq.txt', 'r') as f:
    data = json.load(f)
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

In [18]:
questions[0], answers[0]

('How do I change my password?',
 'After you have logged in, you can change your password using the "Change password" option in the top part of the screen. You have to type your current password and the new password you have chosen in their respective boxes.')

In [22]:
train_questions, temp_questions, train_answers, temp_answers = train_test_split(
    questions, answers, test_size=0.2, random_state=42
)

# Second split: Split the 20% temp into 10% validation and 10% test
val_questions, test_questions, val_answers, test_answers = train_test_split(
    temp_questions, temp_answers, test_size=0.5, random_state=42
)

# Print sizes to verify
print(f"Training set size: {len(train_questions)}")
print(f"Validation set size: {len(val_questions)}")
print(f"Test set size: {len(test_questions)}")

Training set size: 1788
Validation set size: 224
Test set size: 224


In [28]:
from transformers import T5Tokenizer


tokenizer = T5Tokenizer.from_pretrained("t5-small")

max_length = 512
train_encodings = tokenizer(train_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
train_labels = tokenizer(train_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_encodings = tokenizer(val_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_labels = tokenizer(val_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")

In [29]:
batch_size = 8  # Adjust based on your resources
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]},
    {"labels": train_labels["input_ids"]}
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"]},
    {"labels": val_labels["input_ids"]}
)).batch(batch_size)

In [30]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), loss=model.compute_loss)

# Step 7: Fine-tune the model
epochs = 3  # Adjust as needed
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/3


ValueError: in user code:

    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_tf_utils.py", line 1672, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file1yjx6wg3.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "/tmp/__autograph_generated

In [32]:
import json
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

from sklearn.model_selection import train_test_split

# Step 1: Load the data
with open('/content/sample_data/HDFC_Faq.txt', 'r') as f:
    data = json.load(f)
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

# Step 2: Split into train, validation, and test sets
train_questions, temp_questions, train_answers, temp_answers = train_test_split(
    questions, answers, test_size=0.2, random_state=42
)
val_questions, test_questions, val_answers, test_answers = train_test_split(
    temp_questions, temp_answers, test_size=0.5, random_state=42
)

print(f"Training set size: {len(train_questions)}")
print(f"Validation set size: {len(val_questions)}")
print(f"Test set size: {len(test_questions)}")

# Step 3: Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
max_length = 512

# Step 4: Tokenize the data
train_encodings = tokenizer(train_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
train_labels_encodings = tokenizer(train_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_encodings = tokenizer(val_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_labels_encodings = tokenizer(val_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")

# Step 5: Prepare decoder_input_ids and labels
# For teacher forcing: decoder_input_ids is the input (shifted right), labels is the target (shifted left)
train_decoder_input_ids = train_labels_encodings["input_ids"][:, :-1]  # Remove last token
train_labels = train_labels_encodings["input_ids"][:, 1:]              # Remove first token
val_decoder_input_ids = val_labels_encodings["input_ids"][:, :-1]
val_labels = val_labels_encodings["input_ids"][:, 1:]

# Adjust attention masks accordingly
train_decoder_attention_mask = train_labels_encodings["attention_mask"][:, :-1]
val_decoder_attention_mask = val_labels_encodings["attention_mask"][:, :-1]

# Step 6: Create TensorFlow datasets
batch_size = 8
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"],
        "decoder_input_ids": train_decoder_input_ids,
        "decoder_attention_mask": train_decoder_attention_mask
    },
    train_labels
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": val_encodings["input_ids"],
        "attention_mask": val_encodings["attention_mask"],
        "decoder_input_ids": val_decoder_input_ids,
        "decoder_attention_mask": val_decoder_attention_mask
    },
    val_labels
)).batch(batch_size)


Training set size: 1788
Validation set size: 224
Test set size: 224


In [34]:

# Step 7: Load and compile the model
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5))

# Step 8: Train the model
epochs = 1
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


  3/224 [..............................] - ETA: 2:55:54 - loss: 14.5489

KeyboardInterrupt: 

In [35]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [1]:
import json
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from sklearn.model_selection import train_test_split
import time

# Check GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:

# Load data
with open('/content/sample_data/HDFC_Faq.txt', 'r') as f:
    data = json.load(f)
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

# Split data
train_questions, temp_questions, train_answers, temp_answers = train_test_split(
    questions, answers, test_size=0.2, random_state=42
)
val_questions, test_questions, val_answers, test_answers = train_test_split(
    temp_questions, temp_answers, test_size=0.5, random_state=42
)

# Determine optimal max_length
tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenized_lengths = [len(tokenizer.encode(q)) for q in questions] + [len(tokenizer.encode(a)) for a in answers]
max_length = min(max(tokenized_lengths) + 10, 512)
print(f"Using max_length: {max_length}")

# Tokenize
train_encodings = tokenizer(train_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
train_labels_encodings = tokenizer(train_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_encodings = tokenizer(val_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_labels_encodings = tokenizer(val_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")

# Prepare decoder inputs and labels
train_decoder_input_ids = train_labels_encodings["input_ids"][:, :-1]
train_labels = train_labels_encodings["input_ids"][:, 1:]
val_decoder_input_ids = val_labels_encodings["input_ids"][:, :-1]
val_labels = val_labels_encodings["input_ids"][:, 1:]
train_decoder_attention_mask = train_labels_encodings["attention_mask"][:, :-1]
val_decoder_attention_mask = val_labels_encodings["attention_mask"][:, :-1]

# Create datasets
batch_size = 8
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"],
        "decoder_input_ids": train_decoder_input_ids,
        "decoder_attention_mask": train_decoder_attention_mask
    },
    train_labels
)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": val_encodings["input_ids"],
        "attention_mask": val_encodings["attention_mask"],
        "decoder_input_ids": val_decoder_input_ids,
        "decoder_attention_mask": val_decoder_attention_mask
    },
    val_labels
)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Token indices sequence length is longer than the specified maximum sequence length for this model (1213 > 512). Running this sequence through the model will result in indexing errors


Using max_length: 512


In [3]:

# Load and train model
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5))

start_time = time.time()
model.fit(train_dataset, validation_data=val_dataset, epochs=1, verbose=1)
print(f"Training time: {(time.time() - start_time) / 60:.2f} minutes")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


224/224 [==============================] - 213s 763ms/step - loss: 2.5315 - val_loss: 0.9735
Training time: 3.85 minutes


In [4]:

# Save model
model.save_pretrained("/content/fine_tuned_t5_model")


In [7]:

# Generate answer
def generate_answer(question, model, tokenizer):
    inputs = tokenizer(question, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

loaded_model = TFT5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5_model")
print(f"Predicted Answer: {generate_answer('Answer: How do I change my password?', loaded_model, tokenizer)}")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/fine_tuned_t5_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Predicted Answer: Answer: How do I change my password?


In [9]:
import json
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from sklearn.model_selection import train_test_split

# Load data
with open('/content/sample_data/HDFC_Faq.txt', 'r') as f:
    data = json.load(f)
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

# Split data
train_questions, temp_questions, train_answers, temp_answers = train_test_split(questions, answers, test_size=0.2, random_state=42)
val_questions, test_questions, val_answers, test_answers = train_test_split(temp_questions, temp_answers, test_size=0.5, random_state=42)

# Optimize max_length
tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenized_lengths = [len(tokenizer.encode(q)) for q in questions] + [len(tokenizer.encode(a)) for a in answers]
max_length = min(max(tokenized_lengths) + 10, 512)
print(f"Using max_length: {max_length}")

# Tokenize
train_encodings = tokenizer(train_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
train_labels_encodings = tokenizer(train_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_encodings = tokenizer(val_questions, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
val_labels_encodings = tokenizer(val_answers, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")

# Prepare decoder inputs and labels
train_decoder_input_ids = train_labels_encodings["input_ids"][:, :-1]
train_labels = train_labels_encodings["input_ids"][:, 1:]
val_decoder_input_ids = val_labels_encodings["input_ids"][:, :-1]
val_labels = val_labels_encodings["input_ids"][:, 1:]
train_decoder_attention_mask = train_labels_encodings["attention_mask"][:, :-1]
val_decoder_attention_mask = val_labels_encodings["attention_mask"][:, :-1]


Token indices sequence length is longer than the specified maximum sequence length for this model (1213 > 512). Running this sequence through the model will result in indexing errors


Using max_length: 512


In [10]:

# Create datasets
batch_size = 8
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "decoder_input_ids": train_decoder_input_ids, "decoder_attention_mask": train_decoder_attention_mask},
    train_labels
)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "decoder_input_ids": val_decoder_input_ids, "decoder_attention_mask": val_decoder_attention_mask},
    val_labels
)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

# Load and train model
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')])
print(history.history)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/5
224/224 [==============================] - 210s 783ms/step - loss: 1.9400 - val_loss: 0.8261
Epoch 2/5
224/224 [==============================] - 170s 759ms/step - loss: 0.8402 - val_loss: 0.7285
Epoch 3/5
224/224 [==============================] - 170s 759ms/step - loss: 0.7330 - val_loss: 0.6976
Epoch 4/5
224/224 [==============================] - 170s 759ms/step - loss: 0.6923 - val_loss: 0.6785
Epoch 5/5
224/224 [==============================] - 170s 759ms/step - loss: 0.6653 - val_loss: 0.6646
{'loss': [1.9400185346603394, 0.8401607871055603, 0.7330175638198853, 0.6922861337661743, 0.665321409702301], 'val_loss': [0.8260658979415894, 0.7284917831420898, 0.6975992918014526, 0.6785441040992737, 0.6646298766136169]}


In [11]:

# Save model
model.save_pretrained("/content/fine_tuned_t5_model")

# Generate answer
def generate_answer(question, model, tokenizer):
    inputs = tokenizer(question, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [12]:

# Test
loaded_model = TFT5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5_model")
sample_question = "How do I change my password?"
print(f"Predicted Answer: {generate_answer(sample_question, loaded_model, tokenizer)}")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/fine_tuned_t5_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Predicted Answer: Wie kann ich mein Password ändern?


In [2]:
import json
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from sklearn.model_selection import train_test_split

# Configuration
MODEL_NAME = "t5-small"
BATCH_SIZE = 8
MAX_EPOCHS = 15
LEARNING_RATE = 3e-5
PREFIX = "answer: "

# Load and preprocess data
with open('/content/sample_data/HDFC_Faq.txt', 'r') as f:
    data = json.load(f)

# Add task prefixes to questions
questions = [PREFIX + entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

# Split data
train_questions, temp_questions, train_answers, temp_answers = train_test_split(
    questions, answers, test_size=0.2, random_state=42
)
val_questions, test_questions, val_answers, test_answers = train_test_split(
    temp_questions, temp_answers, test_size=0.5, random_state=42
)

# Initialize tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

# Calculate max length
tokenized_lengths = [
    len(tokenizer.encode(text))
    for text in questions + answers
]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Token indices sequence length is longer than the specified maximum sequence length for this model (1213 > 512). Running this sequence through the model will result in indexing errors


In [7]:
max_length = min(max(tokenized_lengths) + 10, 512)

In [9]:

max_length = min(max(tokenized_lengths) + 10,512)  # Add small buffer
max_length = min(max_length, 512)  # Never exceed 512
print(f"Using max sequence length: {max_length}")

# Tokenization function
def tokenize_data(input_texts, target_texts):
    model_inputs = tokenizer(
        input_texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_texts,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="tf"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Prepare datasets
train_encodings = tokenize_data(train_questions, train_answers)
val_encodings = tokenize_data(val_questions, val_answers)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    train_encodings["labels"]
)).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": val_encodings["input_ids"],
        "attention_mask": val_encodings["attention_mask"]
    },
    val_encodings["labels"]
)).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)


Using max sequence length: 512


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:

# Model initialization
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_NAME)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

# Training with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=MAX_EPOCHS,
    callbacks=[early_stopping],
    verbose=1
)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/15
224/224 [==============================] - 208s 744ms/step - loss: 2.1344 - val_loss: 0.7582
Epoch 2/15
224/224 [==============================] - 166s 742ms/step - loss: 0.7847 - val_loss: 0.7118
Epoch 3/15
224/224 [==============================] - 166s 743ms/step - loss: 0.7000 - val_loss: 0.6913
Epoch 4/15
224/224 [==============================] - 166s 743ms/step - loss: 0.6672 - val_loss: 0.6758
Epoch 5/15
224/224 [==============================] - 166s 742ms/step - loss: 0.6496 - val_loss: 0.6608
Epoch 6/15
224/224 [==============================] - 166s 743ms/step - loss: 0.6315 - val_loss: 0.6491
Epoch 7/15
224/224 [==============================] - 167s 745ms/step - loss: 0.6175 - val_loss: 0.6374
Epoch 8/15
224/224 [==============================] - 167s 744ms/step - loss: 0.6069 - val_loss: 0.6268
Epoch 9/15
224/224 [==============================] - 166s 743ms/step - loss: 0.5965 - val_loss: 0.6199
Epoch 10/15
224/224 [==============================] - 166s 743m

In [11]:

# Save model
model.save_pretrained("/content/fine_tuned_t5_hdfc")
tokenizer.save_pretrained("/content/fine_tuned_t5_hdfc")



('/content/fine_tuned_t5_hdfc/tokenizer_config.json',
 '/content/fine_tuned_t5_hdfc/special_tokens_map.json',
 '/content/fine_tuned_t5_hdfc/spiece.model',
 '/content/fine_tuned_t5_hdfc/added_tokens.json')

In [12]:
# Inference function
def generate_answer(question, model, tokenizer):
    # Add task prefix
    prefixed_question = PREFIX + question

    inputs = tokenizer(
        prefixed_question,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [13]:

# Test the model
loaded_model = TFT5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5_hdfc")
loaded_tokenizer = T5Tokenizer.from_pretrained("/content/fine_tuned_t5_hdfc")

# Example usage
test_questions = [
    "How do I change my password?",
    "What are the charges for RTGS?",
    "How long does fund transfer take?"
]

for question in test_questions:
    answer = generate_answer(question, loaded_model, loaded_tokenizer)
    print(f"Q: {question}")
    print(f"A: {answer}\n{'-'*50}")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/fine_tuned_t5_hdfc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Q: How do I change my password?
A: You can change your password by changing your password. You can change your password by changing your password.
--------------------------------------------------
Q: What are the charges for RTGS?
A: You have to pay a minimum of Rs. 1 lakh for RTGS. You can pay a minimum of Rs. 1 lakh for RTGS.
--------------------------------------------------
Q: How long does fund transfer take?
A: You can transfer your funds from your bank account to your bank account. You can transfer funds from your bank account to your bank account.
--------------------------------------------------
